# "Dplyr's across: Replicating within Pandas"
> "Columnwise operations in Pandas - as simply as possible"

- toc: true
- branch: master
- badges: true
- hide_binder_badge: True
- hide_colab_badge: True
- comments: true
- author: Samuel Oranyeli
- categories: [python, Pandas, column-wise operations, dplyr, across]
- hide: false
- search_exclude: true
- metadata_key1: "python Pandas"
- metadata_key2: "column-wise operations"

## **Introduction**

[dplyr](https://dplyr.tidyverse.org/index.html) has the [across](https://dplyr.tidyverse.org/reference/across.html) function, which is meant to make column wise processing easy. Pandas has always supported column-wise operations, and in a relatively simple manner. 

This article aims to replicate solutions in the dplyr [column-wise operations](https://dplyr.tidyverse.org/articles/colwise.html) vignette with Pandas.

Let's load in the relevant libraries

In [1]:
import pandas as pd
import numpy as np
import janitor
import sys

print(" pandas version :", pd.__version__,"\n", 
      "numpy version :", np.__version__, "\n", 
      "python version :", sys.version)

 pandas version : 1.4.4 
 numpy version : 1.22.4 
 python version : 3.9.12 | packaged by conda-forge | (main, Mar 24 2022, 23:23:20) 
[Clang 12.0.1 ]


In [2]:
url = "https://raw.githubusercontent.com/tidyverse/dplyr/main/data-raw/starwars.csv"
starwars = pd.read_csv(url)
starwars.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,sex,gender,homeworld,species,films,vehicles,starships
0,Luke Skywalker,172.0,77.0,blond,fair,blue,19.0,male,masculine,Tatooine,Human,"The Empire Strikes Back, Revenge of the Sith, ...","Snowspeeder, Imperial Speeder Bike","X-wing, Imperial shuttle"
1,C-3PO,167.0,75.0,NaN,gold,yellow,112.0,none,masculine,Tatooine,Droid,"The Empire Strikes Back, Attack of the Clones,...",NaN,NaN
2,R2-D2,96.0,32.0,NaN,"white, blue",red,33.0,none,masculine,Naboo,Droid,"The Empire Strikes Back, Attack of the Clones,...",NaN,NaN
3,Darth Vader,202.0,136.0,none,white,yellow,41.9,male,masculine,Tatooine,Human,"The Empire Strikes Back, Revenge of the Sith, ...",NaN,TIE Advanced x1
4,Leia Organa,150.0,49.0,brown,light,brown,19.0,female,feminine,Alderaan,Human,"The Empire Strikes Back, Revenge of the Sith, ...",Imperial Speeder Bike,NaN


In [3]:
# dplyr : 
# starwars %>% 
#   summarise(across(where(is.character), n_distinct))
(starwars
.select_dtypes('object')
.nunique(dropna=False))

name          87
hair_color    13
skin_color    31
eye_color     15
sex            5
gender         3
homeworld     49
species       38
films         24
vehicles      11
starships     17
dtype: int64

In [4]:
# dplyr :
# starwars %>% 
#   group_by(species) %>% 
#   filter(n() > 1) %>% 
#   summarise(across(c(sex, gender, homeworld), n_distinct))
(starwars
.filter(['species', 'sex', 'gender', 'homeworld'])
.groupby('species')
.pipe(lambda df: df.nunique(dropna = False).loc[df.size().gt(1)])
)

,sex,gender,homeworld
species,,,
Droid,1,2,3
Gungan,1,1,1
Human,2,2,16
Kaminoan,2,2,1
Mirialan,1,1,1
Twi'lek,2,2,1
Wookiee,1,1,1
Zabrak,1,1,2


The [pipe](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.pipe.html) function comes in handy here, and allows reuse of the grouped object multiple times

In [5]:
# dplyr : 
# starwars %>% 
#   group_by(homeworld) %>% 
#   filter(n() > 1) %>% 
#   summarise(across(where(is.numeric), ~ mean(.x, na.rm = TRUE)))
(starwars
.set_index('homeworld')
.select_dtypes('number')
.groupby('homeworld')
.pipe(lambda df: df.mean().loc[df.size().gt(1)])
)

,height,mass,birth_year
homeworld,,,
Alderaan,176.333333,64.000000,43.000000
Corellia,175.000000,78.500000,25.000000
Coruscant,173.666667,50.000000,91.000000
Kamino,208.333333,83.100000,31.500000
Kashyyyk,231.000000,124.000000,200.000000
Mirial,168.000000,53.100000,49.000000
Naboo,175.454545,64.166667,55.000000
Ryloth,179.000000,55.000000,48.000000
Tatooine,169.800000,85.375000,54.644444


In [6]:
# dplyr:
# starwars %>% distinct(across(contains("color")))
starwars.filter(like='color').drop_duplicates()

,hair_color,skin_color,eye_color
0,blond,fair,blue
1,NaN,gold,yellow
2,NaN,"white, blue",red
3,none,white,yellow
4,brown,light,brown
...,...,...,...
79,none,pale,white
81,black,dark,dark
82,brown,light,hazel
84,none,none,black


In [7]:
# dplyr
# starwars %>% count(across(contains("color")), sort = TRUE)
starwars.filter(like='color').value_counts(dropna = False)

hair_color    skin_color  eye_color
brown         light       brown        6
              fair        blue         4
none          grey        black        4
black         dark        brown        3
blond         fair        blue         3
                                      ..
none          dark        brown        1
              green       black        1
auburn, grey  fair        blue         1
none          green       red          1
NaN           white, red  red          1
Length: 67, dtype: int64

Let's look at another solution, from [Stack Overflow](https://stackoverflow.com/questions/63200530/python-pandas-equivalent-to-dplyr-1-0-0-summarizeacross):

In [8]:
url = "https://gist.githubusercontent.com/seankross/a412dfbd88b3db70b74b/raw/5f23f993cd87c283ce766e7ac6b329ee7cc2e1d1/mtcars.csv"
cars = pd.read_csv(url)
cars.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [9]:
# dplyr:
# dat <- group_by(mtcars, cyl) 
# summarize(dat, across(ends_with('p'), sum), across(ends_with('t'), mean))
trim = cars.set_index('cyl').filter(regex = ".+(p|t)$")
dat = trim.groupby('cyl')
dat = {key : dat[key].mean() 
       if key.endswith('t') 
       else dat[key].sum() 
       for key in trim}
pd.concat(dat, axis = 1)

,disp,hp,drat,wt
cyl,,,,
4,1156.5,909,4.070909,2.285727
6,1283.2,856,3.585714,3.117143
8,4943.4,2929,3.229286,3.999214
